# Conda Install Locally
In this *notebook* we will explore how to download *Python packages* locally using *Conda*.<sup>[1](#fnt1), [2](#fnt2)</sup> Why would you want to do this? What if you run your notebooks in *Docker* containers,<sup>[3](#fnt3)</sup> and you do not want to build a *new* Docker image<sup>[3](#fnt3)</sup> for every *new project* but instead just download and *cache* the packages you need to run the notebook locally? This notebook, taken from one of my own personal projects using `Graphviz`,<sup>[4](#fnt4)</sup> shows you one strategy for achieving this.

## Setting Up *PATH Variables
The *majority* problem with attempting to install packages locally is appending to the `PATH`/`PYTHONPATH`<sup>[5](#fnt5), [6](#fnt6)</sup> environment variables.<sup>[7](#fnt7)</sup>

In [ ]:
# libs for manipulating *PATH vars
import os
import platform
import sys

# set conda python version (major and minor)
CONDA_PYTHON_VERSION = '.'.join(platform.python_version_tuple()[:2])

# local conda env
LOCAL_CONDA_ENV = '/home/jovyan/.conda/local-jupyter-env'

# local conda env paths
LOCAL_CONDA_ENV_PATHS = (
    os.path.join(LOCAL_CONDA_ENV, f'lib/python{CONDA_PYTHON_VERSION}.zip'),
    os.path.join(LOCAL_CONDA_ENV, f'lib/python{CONDA_PYTHON_VERSION}'),
    os.path.join(LOCAL_CONDA_ENV, f'lib/python{CONDA_PYTHON_VERSION}/lib-dynload'),
    os.path.join(LOCAL_CONDA_ENV, f'lib/python{CONDA_PYTHON_VERSION}/site-packages')
)

# local conda bin paths
LOCAL_CONDA_BIN_PATH = os.path.join(LOCAL_CONDA_ENV, 'bin')

# local graphviz install path
LOCAL_CONDA_GRAPHVIZ_PATH = os.path.join(
    LOCAL_CONDA_ENV, 
    f'lib/python{CONDA_PYTHON_VERSION}/site-packages/graphviz'
)

## Graphviz Install Control Flow
Now when we run a notebook, we need to *ONLY* download `Graphviz` the first time, and then ignore in subsequent notebook runs.

In [ ]:
# check if graphviz installed
if not os.path.exists(LOCAL_CONDA_GRAPHVIZ_PATH):
    # install with conda if not found (may take a while)
    print('Module graphviz not found. Installing now ... (this may take a while).')
    !conda create -p $LOCAL_CONDA_ENV python=$CONDA_PYTHON_VERSION --yes \
     && conda install \
          -p $LOCAL_CONDA_ENV \
          -c anaconda graphviz python-graphviz pydot --yes

# extend PYTHONPATH for local conda packages
for conda_path in LOCAL_CONDA_ENV_PATHS:
    if conda_path not in sys.path:
        sys.path.append(conda_path)

# extend PATH for local conda binaries
if LOCAL_CONDA_BIN_PATH not in os.environ['PATH']:
    os.environ['PATH'] += os.pathsep + LOCAL_CONDA_BIN_PATH 

# now attempt to import graphviz
import graphviz

# simple graphviz example (ref: https://graphviz.readthedocs.io/en/stable/examples.html#hello-py)
g = graphviz.Digraph('G', filename='hello.gv')
g.edge('Hello', 'World')

# display
g

## References
<span id="fnt1">1: [Conda Official Documentation](https://docs.conda.io/en/latest/)</span>
<br>
<span id="fnt2">2: [Conda - Wikipedia](https://en.wikipedia.org/wiki/Conda_(package_manager))</span>
<br>
<span id="fnt3">3: [Docker overview - Docker Official Documentation](https://docs.docker.com/get-started/overview/)</span>
<br>
<span id="fnt4">4: [Graphviz](https://www.graphviz.org/)</span>
<br>
<span id="fnt5">5: [The Module Search Path - Python Official Documentation](https://docs.python.org/3/tutorial/modules.html#the-module-search-path)</span>
<br>
<span id="fnt6">6: [PYTHONPATH - Python Official Documentation](https://docs.python.org/3/using/cmdline.html#envvar-PYTHONPATH)</span>
<br>
<span id="fnt7">7: [Environment Variable - Wikipedia](https://en.wikipedia.org/wiki/Environment_variable)</span>
<br>